In [ ]:
## https://huggingface.co/transformers/model_doc/bert.html
## https://huggingface.co/transformers/pretrained_models.html

### data_loader 的MyPro加载数据
### bert_ner 中 有模型

### 检测模型使用

In [1]:
from data_loader import MyPro,convert_examples_to_features,create_batch_iter
from bert_ner import Bert_Softmax
# from bert_ner import Bert_Softmax
import torch
from transformers import BertConfig,AdamW

import torch.nn as nn

import torch.optim as optim

from Evalutor import evaluate

In [2]:
from Evalutor import evaluate_bor#evaluate_bor

In [3]:
label_list = ['O', 'B-Nh', 'I-Nh', 'B-NDR', 'I-NDR']
train_iter = create_batch_iter("train",'../multi_data/multi_trainner.txt',12,label_list)

config = BertConfig.from_json_file('../bert-base-chinese/bert_config.json')
bert_softmax = Bert_Softmax(config,5).cuda()

In [4]:
weight = [10.0] * len(label_list)   #生成一维

weight[0] = 1

weight = torch.tensor(weight).cuda()

criterion = nn.NLLLoss(weight,size_average=False)      #定义的损失函数，softmax以后求损失函数

/home/irsdc/songwenhui/anaconda3/lib/python3.7/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [5]:
# param_optimizer = bert_softmax.named_parameters()

# param_optimizer = list(bert_softmax.named_parameters())
# no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']

# optimizer_grouped_parameters = [
#         {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
#         {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]

# optimizer = AdamW(optimizer_grouped_parameters,lr = 0.00001)

In [6]:
### 冻结部分参数

unfreeze_layers = ['layer.11','bert.pooler','classifier.']
 
for name, param in bert_softmax.named_parameters():
    print(name,param.size())

print("*"*30)
print('\n')

for name ,param in bert_softmax.named_parameters():
    param.requires_grad = False
    for ele in unfreeze_layers:
        if ele in name:
            param.requires_grad = True
            break
#验证一下
for name, param in bert_softmax.named_parameters():
    if param.requires_grad:
        print(name,param.size())
 
    


bert.embeddings.word_embeddings.weight torch.Size([21128, 768])
bert.embeddings.position_embeddings.weight torch.Size([512, 768])
bert.embeddings.token_type_embeddings.weight torch.Size([2, 768])
bert.embeddings.LayerNorm.weight torch.Size([768])
bert.embeddings.LayerNorm.bias torch.Size([768])
bert.encoder.layer.0.attention.self.query.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.query.bias torch.Size([768])
bert.encoder.layer.0.attention.self.key.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.key.bias torch.Size([768])
bert.encoder.layer.0.attention.self.value.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.value.bias torch.Size([768])
bert.encoder.layer.0.attention.output.dense.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.output.dense.bias torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.weight torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.bias torch.Size([768])
bert.encoder

In [7]:
#过滤掉requires_grad = False的参数
#optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, bert_softmax.parameters()), lr=0.1)
lr = 0.0001
optimizer = AdamW(filter(lambda p: p.requires_grad, bert_softmax.parameters()), lr=lr,weight_decay = 0.1)

In [8]:
# optimizer = optim.SGD(bert_softmax.parameters(), lr = 0.0001, momentum = 0.9)

In [9]:
batch_size = 8

In [10]:
bert_softmax.train()
for e in range(1):
    count = 0
    if (e+1)%2 == 0:
        lr = lr/2
        optimizer = AdamW(filter(lambda p: p.requires_grad, bert_softmax.parameters()), lr=lr,weight_decay = 0.1)
    
    for step, batch in enumerate(train_iter):
        count += 1
        input_ids, input_mask, segment_ids, label_ids = batch   ##(batch_size,512)
        input_ids = input_ids.cuda()
        input_mask = input_mask.cuda()
        segment_ids = segment_ids.cuda()
        label_ids = label_ids.cuda()
        out = bert_softmax(input_ids,segment_ids,input_mask)  #[batch_size,seq_len+2(512)]
#         break
        out1 = out.view(-1, out.size(2))
        res1 = label_ids[0]
        for i in range(1,len(label_ids)):
            res1 = torch.cat((res1,label_ids[i]))
        loss = criterion(out1,res1)
        loss = loss/len(out)
        
        if count %5 == 0:
            print('loss: ',loss)
            out2 = torch.argmax(out,dim=2)
            batch_precision,batch_recall,batch_f1 = evaluate(out2,label_ids,input_mask)
            print('batch_precision:%.4f  batch_recall:%.4f  batch_f1: %.4f' %(batch_precision,batch_recall,batch_f1)) 
            
            bor_precision, bor_recall,bor_f1 = evaluate_bor(out2,label_ids,input_mask)
            print('bor_precision:%.4f  bor_recall:%.4f  bor_f1: %.4f' %(bor_precision,bor_recall,bor_f1)) 
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(str(e)+'：结束')
        
        

loss:  tensor(640.1537, device='cuda:0', grad_fn=<DivBackward0>)
batch_precision:0.7000  batch_recall:0.0279  batch_f1: 0.0536
bor_precision:0.0000  bor_recall:0.0000  bor_f1: 0.0000
loss:  tensor(325.4315, device='cuda:0', grad_fn=<DivBackward0>)
batch_precision:0.9517  batch_recall:0.5328  batch_f1: 0.6832
bor_precision:0.0741  bor_recall:0.0632  bor_f1: 0.0682
loss:  tensor(172.5672, device='cuda:0', grad_fn=<DivBackward0>)
batch_precision:0.7603  batch_recall:0.8675  batch_f1: 0.8104
bor_precision:0.4737  bor_recall:0.6667  bor_f1: 0.5538
loss:  tensor(85.1098, device='cuda:0', grad_fn=<DivBackward0>)
batch_precision:0.7610  batch_recall:0.9695  batch_f1: 0.8527
bor_precision:0.5149  bor_recall:0.7536  bor_f1: 0.6118
loss:  tensor(73.8272, device='cuda:0', grad_fn=<DivBackward0>)
batch_precision:0.9138  batch_recall:0.9593  batch_f1: 0.9360
bor_precision:0.7882  bor_recall:0.8933  bor_f1: 0.8375
loss:  tensor(25.8160, device='cuda:0', grad_fn=<DivBackward0>)
batch_precision:0.9480 

In [ ]:
# evaluate_bor(out2,label_ids,input_mask)

In [ ]:
## batch_precision:0.9526  batch_recall:0.9731  batch_f1: 0.9628


In [ ]:
# def get_entitys(label_id):
#     valid_pos = torch.nonzero(label_id).squeeze(1)
#     entitys = []
#     #         entitys2 = []
#     flag = valid_pos[0]
#     start = flag
#     for vp in valid_pos[1:]:
#         if vp -1 == flag:
#             end = vp
#         else:
#             entity = [start.item(),(end+1).item()]
#     #         entity = entity.replace(' ','')
#             entitys.append(entity)
#             start = vp

#         flag = vp
#     entity = [start.item(),(end+1).item()]
#     # entity = entity.replace(' ','')
#     entitys.append(entity)   ### 最后一个实体
#     return entitys

# get_entitys(label_ids[0])

# out2 = torch.argmax(out,dim=2)
# get_entitys(out2[0])

In [ ]:
## 保存模型
# len(bert_softmax.state_dict())
PATH = '../model_save/bert_softmax'
torch.save(bert_softmax.state_dict(), PATH)

In [ ]:
### load model
config = BertConfig.from_json_file('../bert-base-chinese/bert_config.json')
# bert_softmax = Bert_Softmax(config,5).cuda()

In [ ]:
label_list = ['O', 'B-Nh', 'I-Nh', 'B-NDR', 'I-NDR']
eval_iter = create_batch_iter("train",'../multi_data/multi_testner.txt',8,label_list)

In [ ]:
### load_model
PATH = '../model_save/bert_softmax'
model = Bert_Softmax(config,5).cuda()
model.load_state_dict(torch.load(PATH))
model.eval()

#### load_model 
# out = model(input_ids,segment_ids,input_mask)  
        
# out1 = out.view(-1, out.size(2))
# res1 = label_ids[0]
# for i in range(1,len(label_ids)):
#     res1 = torch.cat((res1,label_ids[i]))
# loss = criterion(out1,res1)
# loss = loss/len(out)

# if count %5 == 0:
# print('loss: ',loss)
# out2 = torch.argmax(out,dim=2)
# batch_precision,batch_recall,batch_f1 = evaluate(out2,label_ids,input_mask)
# print('batch_precision:%.4f  batch_recall:%.4f  batch_f1: %.4f' %(batch_precision,batch_recall,batch_f1)) 

In [ ]:
weight = [10.0] * len(label_list)   #生成一维

weight[0] = 1

weight = torch.tensor(weight).cuda()

criterion = nn.NLLLoss(weight,size_average=False)      #定义的损失函数，softmax以后求损失函数

In [ ]:
model.eval()

In [ ]:
eval_iter = create_batch_iter("train",'../multi_data/multi_testner.txt',8,label_list)
# bert_softmax.train()
for step, batch in enumerate(eval_iter):
#     count += 1
    input_ids, input_mask, segment_ids, label_ids = batch   ##(batch_size,512)
    input_ids = input_ids.cuda()
    input_mask = input_mask.cuda()
    segment_ids = segment_ids.cuda()
    label_ids = label_ids.cuda()
    out = model(input_ids,segment_ids,input_mask)  #[batch_size,seq_len+2(512)]

    out1 = out.view(-1, out.size(2))
    res1 = label_ids[0]
    for i in range(1,len(label_ids)):
        res1 = torch.cat((res1,label_ids[i]))
    loss = criterion(out1,res1)
    loss = loss/len(out)

    
    print('loss: ',loss)
    out2 = torch.argmax(out,dim=2)
    batch_precision,batch_recall,batch_f1 = evaluate(out2,label_ids,input_mask)
    print('batch_precision:%.4f  batch_recall:%.4f  batch_f1: %.4f' %(batch_precision,batch_recall,batch_f1)) 


In [ ]:
from collections import Counter 

In [ ]:
Counter(input_mask[3].tolist())

In [ ]:
Counter(label_ids[3].tolist())

In [ ]:
Counter(input_ids[3].tolist())

In [ ]:
    #     label_true = []
    #     y_pre = []
#         loss = torch.tensor(0.0,requires_grad=True)
#         for i in range(len(out)):
#     #         label_true.append([label_ids[i][j].item() for j in range(512) if input_mask[i][j] == 1 ])
#     #         y_pre.append([out[i][j].tolist() for j in range(512) if input_mask[i][j] == 1 ])
            
#             label_true = [label_ids[i][j].item() for j in range(512) if input_mask[i][j] == 1 ]
#             y_pre = [out[i][j].tolist() for j in range(512) if input_mask[i][j] == 1 ]

#             assert len(label_true) == len(y_pre)

#             label_true = torch.tensor(label_true).cuda()
#             y_pre = torch.tensor(y_pre).cuda()

#             tmp_loss = criterion(y_pre, label_true)
# #             tmp_loss = tmp_loss/len(y_pre)
#             loss = loss+tmp_loss
# #             loss.backward()
# #             optimizer.backward(loss)
    
    
#         break

##  Pipeline_middle

In [ ]:
from data_loader import MyPro,convert_examples_to_features,create_batch_iter
from bert_ner import Bert_Softmax
# from bert_ner import Bert_Softmax
import torch
from transformers import BertConfig,AdamW

import torch.nn as nn

import torch.optim as optim

from Evalutor import evaluate

In [ ]:
from transformers import BertTokenizer

In [ ]:
label_list = ['O', 'B-Nh', 'I-Nh', 'B-NDR', 'I-NDR']
# train_iter = create_batch_iter("train",'../multi_data/multi_trainner.txt',12,label_list)

config = BertConfig.from_json_file('../bert-base-chinese/bert_config.json')
bert_softmax = Bert_Softmax(config,5).cuda()

In [ ]:
PATH = '../model_save/bert_softmax'
model = Bert_Softmax(config,5).cuda()
model.load_state_dict(torch.load(PATH))
model.eval()

In [ ]:
weight = [10.0] * len(label_list)   #生成一维

weight[0] = 1

weight = torch.tensor(weight).cuda()

criterion = nn.NLLLoss(weight,size_average=False)      #定义

In [ ]:
tokenizer = BertTokenizer('../bert-base-chinese/vocab.txt')

In [ ]:
# eval_iter = create_batch_iter("train",'../multi_data/multi_testner.txt',8,label_list)
# bert_softmax.train()
lines = []
train_iter = create_batch_iter("train",'../multi_data/multi_trainner.txt',12,label_list)
for step, batch in enumerate(train_iter):
#     y_pres = []
#     count += 1
    input_ids, input_mask, segment_ids, label_ids = batch   ##(batch_size,512)
    input_ids = input_ids.cuda()
    input_mask = input_mask.cuda()
    segment_ids = segment_ids.cuda()
    label_ids = label_ids.cuda()
    out = model(input_ids,segment_ids,input_mask)  #[batch_size,seq_len+2(512)]
    
    out = torch.argmax(out,dim= 2)
    
    for i in range(len(out)):
        label_true = [label_ids[i][j].item() for j in range(512) if input_mask[i][j] == 1 ]
        y_pre = [out[i][j].tolist() for j in range(512) if input_mask[i][j] == 1 ]
        y_pre = torch.tensor(y_pre)
        valid_pos = torch.nonzero(y_pre).squeeze(1)
        
        entitys = []
#         entitys2 = []
        flag = valid_pos[0]
        start = flag
        for vp in valid_pos[1:]:
            if vp -1 == flag:
                end = vp
            else:
                entity = tokenizer.decode(input_ids[i][start:end+1])
                entity = entity.replace(' ','')
                entitys.append(entity)
                start = vp
        
            flag = vp
        entity = tokenizer.decode(input_ids[i][start:valid_pos[-1]+1])
        entity = entity.replace(' ','')
        entitys.append(entity)   ### 最后一个实体
#         print(entitys)
        context = tokenizer.decode(input_ids[i][:len(y_pre)][1:-1])
        context = context.replace(' ','')
#         context = context[1:-1]
        
        
        entitys = list(set(entitys))   ### 去重，无所谓顺序，因为两两实体都会进行组合
#         e1_e2 = []
        e_len = len(entitys)
        for i in range(e_len):
            e1 = entitys[i]
            for j in range(e_len):
                if i == j:
                    continue
                e2 = entitys[j]
                line = e1+'，'+e2+'。'+context
                lines.append(line)
#                 print(line)
#                 e1_e2.append([e1,e2])

                
#         print(e1_e2)
#         print(context)
        
        
        
#         y_pres.append()
        
#         break
        
    
    
#     break

    '''out1 = out.view(-1, out.size(2))
    res1 = label_ids[0]
    for i in range(1,len(label_ids)):
        res1 = torch.cat((res1,label_ids[i]))
    loss = criterion(out1,res1)
    loss = loss/len(out)

    
    print('loss: ',loss)
    out2 = torch.argmax(out,dim=2)
    batch_precision,batch_recall,batch_f1 = evaluate(out2,label_ids,input_mask)
    print('batch_precision:%.4f  batch_recall:%.4f  batch_f1: %.4f' %(batch_precision,batch_recall,batch_f1)) '''


In [ ]:
with open('tmp/tmp.txt','r',encoding = 'utf-8') as f:
    lines = f.readlines()
f.close()

In [ ]:
print(len(lines))

In [ ]:
lines = [line for line in lines if len(json.loads(line))<=510]

In [ ]:
print(len(lines))

In [ ]:
re_lines = []
for line in lines:
    t = {}
    t['source'] = json.loads(line)
    t['target'] = 'O'
    re_lines.append(t)

In [ ]:
print(len(re_lines))

In [ ]:
re_lines[-1]

In [ ]:
import json


from tqdm import tqdm
with open('tmp/tmp2.txt','w+',encoding = 'utf-8') as f:
    for line in tqdm(re_lines):
        line_str = json.dumps(line)
        f.write(line_str+'\n')

In [ ]:
print(len(lines))

# BERT-RE

In [1]:
# from redata_loader import MyPro,convert_examples_to_features,create_batch_iter
from redata_loader import create_batch_iter
from bert_ner import RE_Bert_Softmax
# from bert_ner import Bert_Softmax
import torch
from transformers import BertConfig,AdamW

import torch.nn as nn

import torch.optim as optim

import torch.nn.functional as F
from Evalutor import re_evaluate

In [2]:
tag2id = {}
tag2id['O'] = 0
tag2id['traffic_in'] = 1
tag2id['sell_drugs_to'] = 2
tag2id['provide_shelter_for'] = 3
tag2id['posess'] = 4

In [4]:
train_iter = create_batch_iter("train",'../multi_data/multi_trainre.txt',100,tag2id)

In [5]:
config = BertConfig.from_json_file('../bert-base-chinese/bert_config.json')
bert_softmax = RE_Bert_Softmax(config,5).cuda()

In [6]:
# weight = [10.0] * len(tag2id)   #生成一维
# weight[0] = 1
# weight = torch.tensor(weight).cuda()

criterion = nn.NLLLoss(size_average=False)      #定义的损失函数，softmax以后求损失函数

/home/irsdc/songwenhui/anaconda3/lib/python3.7/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [7]:
### 冻结部分参数

unfreeze_layers = ['layer.11','bert.pooler','classifier.']
 
for name, param in bert_softmax.named_parameters():
    print(name,param.size())

print("*"*30)
print('\n')

for name ,param in bert_softmax.named_parameters():
    param.requires_grad = False
    for ele in unfreeze_layers:
        if ele in name:
            param.requires_grad = True
            break
#验证一下
for name, param in bert_softmax.named_parameters():
    if param.requires_grad:
        print(name,param.size())

bert.embeddings.word_embeddings.weight torch.Size([21128, 768])
bert.embeddings.position_embeddings.weight torch.Size([512, 768])
bert.embeddings.token_type_embeddings.weight torch.Size([2, 768])
bert.embeddings.LayerNorm.weight torch.Size([768])
bert.embeddings.LayerNorm.bias torch.Size([768])
bert.encoder.layer.0.attention.self.query.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.query.bias torch.Size([768])
bert.encoder.layer.0.attention.self.key.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.key.bias torch.Size([768])
bert.encoder.layer.0.attention.self.value.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.value.bias torch.Size([768])
bert.encoder.layer.0.attention.output.dense.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.output.dense.bias torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.weight torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.bias torch.Size([768])
bert.encoder

In [8]:
#过滤掉requires_grad = False的参数
#optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, bert_softmax.parameters()), lr=0.1)
lr = 0.0002
optimizer = AdamW(filter(lambda p: p.requires_grad, bert_softmax.parameters()), lr=lr,weight_decay = 0.1)

In [9]:
bert_softmax.train()
for e in range(8):
    count = 0
    if (e+1)%2 == 0:
        lr = lr/2
        optimizer = AdamW(filter(lambda p: p.requires_grad, bert_softmax.parameters()), lr=lr,weight_decay = 0.1)
    
    for step, batch in enumerate(train_iter):
        count += 1
        input_ids, input_mask, segment_ids, label_ids = batch   ##(batch_size,512)
        input_ids = input_ids.cuda()
        input_mask = input_mask.cuda()
        segment_ids = segment_ids.cuda()
        label_ids = label_ids.cuda()
        out = bert_softmax(input_ids,segment_ids,input_mask)  #[batch_size,seq_len+2(512)]
        
        loss = F.cross_entropy(out, label_ids)
#         break
        
#         out1 = out.view(-1, out.size(2))
#         res1 = label_ids[0]
#         for i in range(1,len(label_ids)):
#             res1 = torch.cat((res1,label_ids[i]))
#         loss = criterion(out,label_ids)
        #loss = loss/len(out)
        
        if count %5 == 0:
            print('loss: ',loss)
#             out2 = torch.argmax(out,dim=2)
            batch_precision,batch_recall,batch_f1 = re_evaluate(out,label_ids)
            print('batch_precision:%.4f  batch_recall:%.4f  batch_f1: %.4f' %(batch_precision,batch_recall,batch_f1)) 
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(str(e)+'：结束')

loss:  tensor(1.4221, device='cuda:0', grad_fn=<NllLossBackward>)
batch_precision:0.4595  batch_recall:0.2931  batch_f1: 0.3579
loss:  tensor(1.4726, device='cuda:0', grad_fn=<NllLossBackward>)
batch_precision:0.0000  batch_recall:0.0000  batch_f1: 0.0000
loss:  tensor(1.2452, device='cuda:0', grad_fn=<NllLossBackward>)
batch_precision:0.6000  batch_recall:0.3000  batch_f1: 0.4000
loss:  tensor(1.0295, device='cuda:0', grad_fn=<NllLossBackward>)
batch_precision:0.6429  batch_recall:0.4655  batch_f1: 0.5400
loss:  tensor(0.9801, device='cuda:0', grad_fn=<NllLossBackward>)
batch_precision:0.5882  batch_recall:0.4918  batch_f1: 0.5357
loss:  tensor(0.8568, device='cuda:0', grad_fn=<NllLossBackward>)
batch_precision:0.7333  batch_recall:0.8209  batch_f1: 0.7746
loss:  tensor(0.8205, device='cuda:0', grad_fn=<NllLossBackward>)
batch_precision:0.7042  batch_recall:0.7812  batch_f1: 0.7407
0：结束
loss:  tensor(0.6940, device='cuda:0', grad_fn=<NllLossBackward>)
batch_precision:0.7969  batch_rec

In [ ]:
## 1.00 0.8642 0.9167

In [ ]:
## 保存模型
# len(bert_softmax.state_dict())
PATH = '../model_save/re_bert_softmax'
torch.save(bert_softmax.state_dict(), PATH)

## pipeline 之RE

In [ ]:
# from redata_loader import MyPro,convert_examples_to_features,create_batch_iter
from redata_loader import create_batch_iter
from bert_ner import RE_Bert_Softmax
# from bert_ner import Bert_Softmax
import torch
from transformers import BertConfig,AdamW

import torch.nn as nn

import torch.optim as optim

import torch.nn.functional as F
from Evalutor import re_evaluate

In [ ]:
from transformers import BertTokenizer

In [ ]:
tag2id = {}
tag2id['O'] = 0
tag2id['traffic_in'] = 1
tag2id['sell_drugs_to'] = 2
tag2id['provide_shelter_for'] = 3
tag2id['posess'] = 4

In [ ]:
id2tag = []
id2tag.append('O')
id2tag.append('traffic_in')
id2tag.append('sell_drugs_to')
id2tag.append('provide_shelter_for')
id2tag.append('posess')

In [ ]:
train_iter = create_batch_iter("train",'tmp/tmp2.txt',16,tag2id)

In [ ]:
config = BertConfig.from_json_file('../bert-base-chinese/bert_config.json')
bert_softmax = RE_Bert_Softmax(config,5).cuda()

In [ ]:
PATH = '../model_save/re_bert_softmax'
rmodel = RE_Bert_Softmax(config,5).cuda()
rmodel.load_state_dict(torch.load(PATH))
rmodel.eval()

In [ ]:
tokenizer = BertTokenizer('../bert-base-chinese/vocab.txt')

In [ ]:
res_lines = []
for step, batch in enumerate(train_iter):
#     count += 1
    input_ids, input_mask, segment_ids, label_ids = batch   ##(batch_size,512)
    input_ids = input_ids.cuda()
    input_mask = input_mask.cuda()
    segment_ids = segment_ids.cuda()
    label_ids = label_ids.cuda()
    out = rmodel(input_ids,segment_ids,input_mask)  #[batch_size,seq_len+2(512)]
    out = torch.softmax(out,dim=1)
    out = torch.argmax(out,dim = 1)
    for i in range(len(out)):
        label_true = [1 for j in range(512) if input_mask[i][j] == 1 ] ## 检验有效长度
        length = len(label_true)
        tag = id2tag[out[i]]
        context = tokenizer.decode(input_ids[i][1:length-1])
        context = context.replace(' ','')
        res_line = {}
        res_line['source'] = context
        res_line['target'] = tag
        
        res_lines.append(res_line)
#     break

In [ ]:
len(res_lines)

In [ ]:
import json

from tqdm import tqdm
with open('tmp/result.txt','w+',encoding = 'utf-8') as f:
    for line in tqdm(res_lines):
        line_str = json.dumps(line)
        f.write(line_str+'\n')

### pipeline 最后预测

In [ ]:
import json

In [ ]:
from transformers import BertTokenizer

In [ ]:
from Evalutor import pipe_evaluate

In [ ]:
prepath = '../multi_data/'
filename = 'multi_train.txt'

In [ ]:
with open(prepath+filename,'r',encoding='gbk') as f:
    lines = f.readlines()
f.close()

In [ ]:
len(lines)

In [ ]:
tokenizer = BertTokenizer('../bert-base-chinese/vocab.txt')

In [ ]:
# ner_lines = []
# re_lines = []
# labels = []
true_infor = {}
for line in lines:
    ner_line = {}
    con = json.loads(line)
    sentText = con['sentText']
    relationMentions = con['relationMentions']
    triples = []
    tuples = []
    entitys = []
#     e1s = []
#     e2s = []
    for relationMention in relationMentions:
        e1 = relationMention['em1Text']
        e2 = relationMention['em2Text']
        label = relationMention['label']
        triples.append((label,e1,e2))
        tuples.append((e1,e2))
        entitys.append(e1)
        entitys.append(e2)
        
    senttokens = list(sentText)
    input_ids = tokenizer.convert_tokens_to_ids(senttokens)
    sentText = tokenizer.decode(input_ids)
    sentText = sentText.replace(' ','')
    true_infor[sentText] = triples
#     break
#     break

In [ ]:
len(true_infor)

In [ ]:
with open('tmp/result.txt','r',encoding='gbk') as f:
    mlines = f.readlines()
f.close()

In [ ]:
len(mlines)

In [ ]:
etotal = [0.0,0.0,0.0,0.0]  ### tp,tn,fp,fn
for line in mlines:
    line_dict = json.loads(line)
    source = line_dict['source']
    label = line_dict['target']
    so_list = source.split('。',1)
    so1 = so_list[1].replace('[[UNK][UNK][UNK]]','[UNK]')
    
    if so1 not in true_infor:
        continue
    Tlabels = true_infor[so1]
#     print(Tlabels)   ### e.g. [('posess', '欧某某', '甲基苯丙胺'), ('posess', '欧某某', '麻黄碱'), ('posess', '欧某某', '巴比妥')]
    
    so_0 = so_list[0]
    e1_e2 = so_0.split('，')
    pre_infor = (label,e1_e2[0],e1_e2[1])
#     print(pre_infor)   ###e.g. ('posess', '欧某某', '甲基苯丙胺')
    
    eva = pipe_evaluate(pre_infor,Tlabels)
    etotal = [ii+jj for ii,jj in zip(etotal,eva)]
#     print(etotal)
#     break
    
#     aa = aa.replace('[[UNK][UNK][UNK]]','[UNK]')
#     if so1 not in true_infor:
#         print(so1)
    
    
#     break


In [ ]:
tp,tn,fp,fn = etotal
p = tp/(tp+fp)
r = tp/(tp+fn)
f1 = 2*p*r/(p+r)
print('precision: ',tp/(tp+fp))   ## 0.2984
print('recall: ',tp/(tp+fn))    ## 0.9885
print('f1: ',f1)                ## 0.4542

In [ ]:
tp

In [ ]:
fp

In [ ]:
tp/(tp+fp)

In [ ]:
tp/(tp+fn)

In [ ]:
for k,v in true_infor.items():
    if k[:9] == '同年11月3日10时许，':
        k1 = k
        print(k)